In [2]:
%pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git (to revision a1d071733d7111c9c014f024669f959182114e33) to /private/var/folders/4_/051f5l115s33r4zvqhcsst0r0000gn/T/pip-install-_8jc7wb7/clip_31378a0dad594d179af4de51c841b944
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/4_/051f5l115s33r4zvqhcsst0r0000gn/T/pip-install-_8jc7wb7/clip_31378a0dad594d179af4de51c841b944
  Running command git rev-parse -q --verify 'sha^a1d071733d7111c9c014f024669f959182114e33'
  Running command git fetch -q https://github.com/openai/CLIP.git a1d071733d7111c9c014f024669f959182114e33
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the 

In [8]:
import pandas 

df = pandas.read_csv('./mincult-train/train.csv', sep=';')
df = df[:100]
df['path'] = './mincult-train/train/'+ df['object_id'].astype(str) + '/' + df['img_name']
df['embedding_path'] = df['path']+'.embedding'

In [3]:
df.head()

,object_id,name,description,group,img_name,path,embedding_path
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg,./mincult-train/train/10669820/7862029.jpg,./mincult-train/train/10669820/7862029.jpg.emb...
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg,./mincult-train/train/4489444/9461061.jpg,./mincult-train/train/4489444/9461061.jpg.embe...
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg,./mincult-train/train/8722586/5095122.jpg,./mincult-train/train/8722586/5095122.jpg.embe...
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg,./mincult-train/train/3712248/551422.jpg,./mincult-train/train/3712248/551422.jpg.embed...
4,6339754,Сабля.,NaN,Оружие,2592073.jpg,./mincult-train/train/6339754/2592073.jpg,./mincult-train/train/6339754/2592073.jpg.embe...


In [4]:
from PIL import Image
import clip
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device}")
device = torch.device(device)
model, preprocess = clip.load("ViT-L/14")
model.to(device)

using cpu


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


In [5]:
def get_embedding(image_path): 
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        return model.encode_image(image)[0].tolist()

In [9]:
import json
GEN_EMBEDDING = True
if GEN_EMBEDDING: # if we need to generate embeddings from scratch
    for index, row in df.iterrows(): 
        embedding = get_embedding(row.path)
        with open(row.embedding_path, 'w') as f:
            f.write(json.dumps(embedding))

In [ ]:
from scipy.spatial import distance

def find_best(image_path): 
    emb = get_embedding(image_path) 
    distances = {} 
    for index, row in df.iterrows(): 
        with open(row.embedding_path, 'r') as f:
            other_emb = json.loads(f.readline())
            dist = distance.cosine(emb, other_emb)
            distances[index] = dist
    dists = sorted(list(distances.items()), key=lambda a: a[1])[:10] 
    for i, dist in dists: 
        print(df.loc[[i]].path)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [1]:
find_best('swordjpg')

NameError: name 'find_best' is not defined

In [4]:
import ruclip 
model_name = "ruclip-vit-base-patch32-224"
clip, processor = ruclip.load(model_name, device="cuda")

OSError: Distant resource does not have an ETag, we won't be able to reliably ensure reproducibility.